In [11]:
import requests
from requests.structures import CaseInsensitiveDict
import folium
from typing import Literal

API_KEY = "25fa34886b044263aa56dcc148c94abc"
BASE_URL = "https://api.geoapify.com/v1/routing"

ENGINE_TYPES = Literal["petrol", "diesel", "electric", "hybrid"]

In [12]:

def get_optimized_route(waypoints, vehicle_type="drive"):
    waypoints_str = "%7C".join([f"{lat},{lon}" for lat, lon in waypoints])
    url = f"{BASE_URL}?waypoints={waypoints_str}&mode={vehicle_type}&optimize=true&apiKey={API_KEY}"
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        features = data.get('features', [])
        if features:
            route = features[0].get('properties', {})
            distance = route.get('distance', 0)  # In meters
            geometry = features[0].get('geometry', {}).get('coordinates', [])
            return distance / 1000, geometry  # Convert to kilometers and return geometry
        else:
            print("No route found.")
            return 0, None
    else:
        print(f"Error: {response.status_code}")
        return 0, None

In [13]:

def calculate_emissions(engine_type:ENGINE_TYPES, fuel_efficiency, distance):
    fuel_consumed = fuel_efficiency * distance
    emissions = 0.0
    if engine_type == "petrol":
        emissions = 2.31 * fuel_consumed
    elif engine_type == "diesel":
        emissions = 2.68 * fuel_consumed
    elif engine_type == "electric":
        emissions = 0.2 * (fuel_consumed / 1000)
    elif engine_type == "hybrid":
        petrol_usage = fuel_consumed * 0.5
        electric_usage = fuel_consumed * 0.5
        emissions = (2.31 * petrol_usage) + (0.2 * electric_usage / 1000)
    else:
        print("Invalid engine type.")
        return None
    return emissions

In [14]:

def calculate_shared_delivery_emissions(driver_location, restaurants, customers, engine_type:ENGINE_TYPES, fuel_efficiency):
    waypoints = [driver_location] + restaurants + customers
    distance, geometry = get_optimized_route(waypoints, vehicle_type="drive")
    if distance:
        emissions = calculate_emissions(engine_type, fuel_efficiency, distance)
        return emissions, geometry
    else:
        return 0, None

In [15]:

def flatten_geometry(geometry):
    # Flatten nested structures in geometry, if applicable
    flat_coords = []
    for coord in geometry:
        if isinstance(coord[0], list):
            for sub_coord in coord:
                flat_coords.append(sub_coord)
        else:
            flat_coords.append(coord)
    return flat_coords


In [19]:

def plot_route_on_map(geometry, waypoints):
    # Create a map centered around the driver's location
    route_map = folium.Map(location=waypoints[0], zoom_start=13)
    
    # Add waypoints to the map
    for lat, lon in waypoints:
        folium.Marker(location=[lat, lon]).add_to(route_map)
    
    # Flatten the geometry and plot the route
    if geometry:
        flat_geometry = flatten_geometry(geometry)
        folium.PolyLine(locations=[[lat, lon] for lon, lat in flat_geometry], color="blue", weight=2.5).add_to(route_map)
    
    # Save map as an HTML file
    route_map.save("maps/route_map.html")

In [22]:
# Example usage
driver_location = (1.423206, 103.838137)
restaurants = [
    (1.428627, 103.835870),
    (1.428860, 103.835901)
]
customers = [
    (1.434898, 103.836053),
    (1.429243, 103.828417)
]
engine_type:ENGINE_TYPES = "diesel"
fuel_efficiency = 0.07

total_emissions, geometry = calculate_shared_delivery_emissions(driver_location, restaurants, customers, engine_type, fuel_efficiency)

if total_emissions:
    print(f"Total CO2 Emissions for shared delivery: {total_emissions:.2f} kg")
    
    # Plot the route on a map
    waypoints = [driver_location] + restaurants + customers
    plot_route_on_map(geometry, waypoints)

Total CO2 Emissions for shared delivery: 1.47 kg
